<a href="https://colab.research.google.com/github/mgozon/Tutor-Match/blob/master/TutorMatch_Marcus_Gozon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tutor Match

This program will take in a list of tutors and students (names, classes, times), output the possible matches, and gives a possible maximal matching. Comment in or out the shuffle to get different matchings.

You may want to create a systematic way of assigning numbers to each class. Fix the dictionary in the cpp file.

#Program

In [ ]:
%%writefile tutormatch.cpp

#include <iostream>
#include <vector>
#include <string>
#include <fstream>
#include <algorithm>
#include <climits>
#include <cstdlib>
#include <cstdio>
#include <set>
#include <unordered_set>
#include <map>
#include <unordered_map>
#include <bitset>
#include <deque>
#include <queue>
#include <tuple>
#include <cmath>
#include <cctype>
#include <stack>
#include <random>
#include <chrono>

using namespace std;

typedef long long ll;
typedef vector<int> vi;
typedef pair<int, int> pi;
typedef vector<pair<int, int>> vii;

unordered_map<string, int> classMap ({
    { "ALG1", 01},
    { "GEO", 02},
    { "ALG2", 03},
    { "PCALC", 04},
    { "CALC", 05},
    { "STATS", 06},
    { "CSA", 07},
    { "MAD", 8},

    { "BIO", 11},
    { "EARTH", 12},
    { "CHEM1", 13},
    { "APES", 14},
    { "APCHEM", 15},
    { "APBIO", 16},
    { "APPHYS", 17},
    { "PHYS1", 18},

    { "WORLD", 21},
    { "USH", 22},
    { "ECON", 23},
    { "GOVT", 24},
    { "PSYCH", 25},
    { "HUMANITIES", 26},

    { "ENG9", 31},
    { "ENG10", 32},
    { "APLANG", 33},
    { "APLIT", 34},
    { "ALIT", 35},
    { "CWL", 36}
});

unordered_map<int, string> rclassMap;

struct tp {
    string name;
    vector<int> sub;
    vector<int> times;
};

tp trs[1000]; // tutor info
tp sts[1000]; // student info
int n, m; // tutor count, student count
vector<pair<int, int>> adj[1000];
unordered_map<int, int> idx[1000];
ifstream fin("tutor.in");
ofstream fout("tutor.out");

bool vis[1000] = {false};
deque<int> path;
bool found = false;

vector<int> share(vector<int>& u, vector<int>& v) {
    vector<int> res;
    for (int a = 0, b = 0; a < u.size() && b < v.size();) {
        if (u[a] == v[b]) { res.push_back(u[a]); a++; b++; }
        else if (u[a] < v[b]) a++;
        else b++;
    }
    return res;
}

void debugInput() {
    cout << "Testing input data" << endl;
    cout << "Tutors" << endl;
    for (int i = 0; i < n; i++) {
        cout << i << ": " << trs[i].name << endl;
        for (auto x : trs[i].sub) cout << rclassMap[x] << " "; cout << endl;
        for (auto x : trs[i].times) cout << x << " "; cout << endl;
        cout << endl;
    }
    cout << "Students" << endl;
    for (int i = 0; i < m; i++) {
        cout << i << ": " << sts[i].name << endl;
        for (auto x : sts[i].sub) cout << rclassMap[x] << " "; cout << endl;
        for (auto x : sts[i].times) cout << x << " "; cout << endl;
        cout << endl;
    }
}

void debugGraph() {
    cout << "Validating graph" << endl;
    for (int i = 0; i <= n+m+1; i++) {
        cout << i << ": ";
        for (auto x : adj[i]) cout << "(" << x.first << ", " << x.second << ") ";
        cout << endl;
    }
    cout << endl << "Graph by Names" << endl;
    for (int i = 1; i <= n; i++) {
        cout << trs[i-1].name << ": ";
        for (auto x : adj[i]) 
            if (x.first != 0)
                cout << "(" << sts[x.first-n-1].name << ", " << x.second << ") ";
        cout << endl;
    }
    for (int i = n+1; i <= n+m; i++) {
        cout << sts[i-n-1].name << ": ";
        for (auto x : adj[i])
            if (x.first != n+m+1)
                cout << "(" << trs[x.first-1].name << ", " << x.second << ") ";
        cout << endl;
    }
}

void writeConnections(ostream& pout) {
    pout << "Tutors: \n" << endl;
    for (int i = 0; i < n; i++) {
        pout << trs[i].name << ": " << endl;
        for (auto x : adj[i+1])
            if (x.first >= n+1 && x.first <= n+m) {
                int j = x.first-n-1;
                pout << sts[j].name << " (";
                vector<int> ss = share(trs[i].sub, sts[j].sub);
                for (int x = 0; x < ss.size()-1; x++) pout << rclassMap[ss[x]] << ", ";
                pout << rclassMap[ss.back()] << ") ";
                vector<int> ts = share(trs[i].times, sts[j].times);
                pout << "(";
                for (int x = 0; x < ts.size()-1; x++) pout << ts[x] << ", ";
                pout << ts.back() << ")" << endl;
            }

        pout << endl;
    }
    pout << "Students: \n" << endl;
    for (int j = 0; j < m; j++) {
        pout << sts[j].name << ": " << endl;
        for (auto x : adj[1+n+j])
            if (x.first >= 1 && x.first <= n) {
                int i = x.first-1;
                vector<int> ss = share(trs[i].sub, sts[j].sub);
                pout << trs[i].name << " (";
                for (int x = 0; x < ss.size()-1; x++) pout << rclassMap[ss[x]] << ", ";
                pout << rclassMap[ss.back()] << ") ";
                vector<int> ts = share(trs[i].times, sts[j].times);
                pout << "(";
                for (int x = 0; x < ts.size()-1; x++) pout << ts[x] << ", ";
                pout << ts.back() << ")" << endl;
            }
        pout << endl;
    }
}

void dfs(int u) {
    if (vis[u] || found) return;
    if (u==n+m+1) {
        found = true;
        return;
    }
    vis[u] = true;
    for (auto x : adj[u]) {
        if (x.second == 0) continue;
        path.push_back(x.first);
        dfs(x.first);
        if (found) break;
        path.pop_back();
    }
    vis[u] = false;
}

void writeMaximalMatch(ostream& pout, int num) {
    pout << "We can match at most " << num << " students" << endl;
    pout << "Tutors: " << endl;
    for (int i = 1; i <= n; i++) {
        pout << trs[i-1].name << ": ";
        for (auto x : adj[i]) { // we will print out all matches to debug
            if (x.first == 0 || x.second == 1) continue;
            pout << sts[x.first-n-1].name << " ";
        }
        pout << endl;
    }
    pout << endl << "Students: " << endl;
    for (int i = n+1; i <= n+m; i++) {
        pout << sts[i-n-1].name << ": ";
        for (auto x : adj[i]) { // again we print out all matches to debug
            if (x.first == n+m+1 || x.second == 0) continue;
            pout << trs[x.first-1].name << " ";
        }
        pout << endl;
    }
    pout << endl;
}

void divide(ostream& pout) {
    pout << "----------------------------------------" << endl;
}

int main() {
    
    for (auto x : classMap) {
        rclassMap[x.second] = x.first;
    }

    // read input data
    fin >> n;
    for (int i = 0; i < n; i++) {
        string name;
        fin.ignore(100, '\n');
        getline(fin, name);
        name.erase(remove(name.begin(), name.end(), '\r'), name.end());
        name.erase(remove(name.begin(), name.end(), '\n'), name.end());
        trs[i].name = name;
        int cls, tms;
        fin >> cls >> tms;

        string cn;
        for (int j = 0; j < cls; j++) {
            fin >> cn;
            trs[i].sub.push_back(classMap[cn]);
        }

        int tm;
        for (int j = 0; j < tms; j++) {
            fin >> tm;
            trs[i].times.push_back(tm);
        }

        sort(trs[i].sub.begin(), trs[i].sub.end());
        sort(trs[i].times.begin(), trs[i].times.end());
    }

    fin >> m;
    for (int i = 0; i < m; i++) {
        string name;
        fin.ignore(100, '\n');
        getline(fin, name);
        name.erase(remove(name.begin(), name.end(), '\r'), name.end());
        name.erase(remove(name.begin(), name.end(), '\n'), name.end());
        sts[i].name = name;
        int cls, tms;
        fin >> cls >> tms;

        string cn;
        for (int j = 0; j < cls; j++) {
            fin >> cn;
            sts[i].sub.push_back(classMap[cn]);
        }

        int tm;
        for (int j = 0; j < tms; j++) {
            fin >> tm;
            sts[i].times.push_back(tm);
        }

        sort(sts[i].sub.begin(), sts[i].sub.end());
        sort(sts[i].times.begin(), sts[i].times.end());
    }

    // verify data transfer
    debugInput();

    // *OPTIONAL shuffle data for different maximal matchings
    mt19937 rng(chrono::steady_clock::now().time_since_epoch().count());
    shuffle(trs, trs+n, rng);
    shuffle(sts, sts+m, rng);

    // build graph (connections represent matches between tutors and students)
    // 0 = source, [1, n] = tutors, [n+1, n+m] = students, n+m+1 = sink
    for (int i = 1; i <= n; i++) {
        adj[0].push_back({i, 1});
        adj[i].push_back({0, 0});
        idx[0][i] = adj[0].size()-1;
        idx[i][0] = adj[i].size()-1;
    }

    for (int i = n+1; i <= n+m; i++) {
        adj[i].push_back({n+m+1, 1});
        adj[n+m+1].push_back({i, 0});
        idx[i][n+m+1] = adj[i].size()-1;
        idx[n+m+1][i] = adj[n+m+1].size()-1;
    }

    for (int i = 0; i < n; i++) {
        for (int j = 0; j < m; j++) {
            if (share(trs[i].sub, sts[j].sub).size() == 0) continue;
            if (share(trs[i].times, sts[j].times).size() == 0) continue;

            adj[1+i].push_back({1+n+j, 1});
            adj[1+n+j].push_back({1+i, 0});
            idx[1+i][1+n+j] = adj[1+i].size()-1;
            idx[1+n+j][1+i] = adj[1+n+j].size()-1;
        }
    }

    // verify connections
    debugGraph();

    // print matching data
    writeConnections(fout);
    divide(fout);

    // Finally the optimal matching algorithm
    // Keep on running dfs to find more paths and update the graph until no more
    int best = 0;
    while (true) {
        //memset(vis, false, n+m+2);
        path.clear();
        found = false;
        dfs(0);
        if (found == false) break;

        int src = 0, next = path.front();
        path.pop_front();
        while (true) {
            adj[src][idx[src][next]].second--;
            adj[next][idx[next][src]].second++;

            if (path.size() == 0) break;
            src = next;
            next = path.front();
            path.pop_front();
        }
        best++;
    }

    writeMaximalMatch(fout, best);

    cout << "Here is what your document will look like: " << endl;
    writeConnections(cout);
    divide(cout);
    writeMaximalMatch(cout, best);

    return 0;
}

Writing tutormatch.cpp


#Upload Input File

They should be in the following format, with an example below:

In [ ]:
%%writefile inputformat.in

(# tutors)
name1
(# classes) (# times)
class1 ... class(classes)
time1 ... time(times)
name 2
...
(#students)
name1
(# classes) (#times)
class1 ... class(classes)
time1 ... time(times)
name2 ...

Writing inputformat.in


In [ ]:
%%writefile dinput.in

1
Marcus Gozon
2 2
CALC CSA
2 3
2
Arnav Brahmasandra
2 3
CALC APBIO
10 2 3
Charlie Clynes
1 1
CALC
3

Writing dinput.in


In [ ]:
from google.colab import files
!rm tutor.in
files.upload()
%ls

rm: cannot remove 'tutor.in': No such file or directory


Saving tutor.in to tutor.in
sample_data/  tutor.in


#Run Program

In [ ]:
%%script bash

g++ tutormatch.cpp -o name
ls -laX
./name

total 264
-rwxr-xr-x 1 root root 215016 Oct 18 21:32 name
drwxr-xr-x 1 root root   4096 Oct 14 16:31 sample_data
drwxr-xr-x 1 root root   4096 Oct 18 21:32 .
drwxr-xr-x 1 root root   4096 Oct 18 19:16 ..
drwxr-xr-x 1 root root   4096 Oct 14 16:32 .config
-rw-r--r-- 1 root root   9752 Oct 18 21:08 tutormatch.cpp
-rw-r--r-- 1 root root    101 Oct 18 19:46 dinput.in
-rw-r--r-- 1 root root    195 Oct 18 19:51 inputformat.in
-rw-r--r-- 1 root root    468 Oct 18 21:31 tutor.in
-rw-r--r-- 1 root root    117 Oct 18 20:52 tutor (1).out
-rw-r--r-- 1 root root   1080 Oct 18 21:31 tutor.out
-rw-r--r-- 1 root root    113 Oct 18 20:52 tutor.in.txt
Testing input data
Tutors
0: Yonglu (Lulu) Zhang
ALG1 GEO ALG2 PCALC BIO EARTH ENG9 ENG10 
2 3 

1: Kaitlin McCreadie
ALG1 ALG2 EARTH WORLD USH ENG9 ENG10 
10 11 12 

2: Zach Pryor
ALG1 ALG2 PCALC 
1 12 

3: Kyle Cheng
ALG1 GEO ECON ENG9 ENG10 
2 3 

4: Marcus Gozon
CALC CSA APPHYS 
1 2 3 4 

Students
0: Ashton Odegaard
ALG1 BIO WORLD ENG9 
1 2 3 4 

1: St

#Download Results

In [ ]:
from google.colab import files
files.download("tutor.out")

FileNotFoundError: ignored

In [ ]:
5
Yonglu (Lulu) Zhang
8 2
ALG1 GEO ALG2 PCALC BIO EARTH ENG9 ENG10
2 3
Kaitlin McCreadie
7 3
ALG1 ALG2 EARTH WORLD USH ENG9 ENG10
10 11 12
Zach Pryor
3 2
ALG1 ALG2 PCALC
12 1
Kyle Cheng
5 2
ALG1 GEO ECON ENG9 ENG10
2 3
Marcus Gozon
3 4
CALC CSA APPHYS
1 2 3 4
4
Ashton Odegaard
4 4
ALG1 BIO WORLD ENG9
1 2 3 4
Stanley Ye
2 4
CALC ECON
2 3 4 5
Rida Albarkat 
4 3
GEO BIO WORLD ENG9
10 11 12
Neisja Brown
3 2
ALG1 BIO WORLD
1 3


